In [1]:
from platform import python_version
print(python_version())

3.12.4


In [2]:
import xlwings as xw
import numpy as np
import pandas as pd
import os, os.path
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from outliers import smirnov_grubbs as grubbs

In [3]:
os.getcwd()

'd:\\github\\mitodyn-data\\1229_PANC1_tox'

In [6]:
def formatpv(pv, threshold=1E-4):
    if pv > threshold:
        return str(f'P={pv:.4f}')
    else:
        return str(f'P={pv:.2E}')

## Plot Function

In [17]:
def Plot4_seaborn(in_data, name, unit, group, savedir='boxplot'):
    mydata = in_data
    # data = [temp_data_0X, temp_data_025X, temp_data_05X, temp_data_1X, temp_data_2X]
    # data = [grubbs.test(in_data[0], alpha=0.05), grubbs.test(in_data[1], alpha=0.05),
    #         grubbs.test(in_data[2], alpha=0.05), grubbs.test(in_data[3], alpha=0.05)]
    max_value = np.amax([np.amax(mydata[0]), np.amax(mydata[1]), np.amax(mydata[2]), np.amax(mydata[3])])
    numBoxes = 4
    fig1, ax1 = plt.subplots(figsize=(6, 9), dpi=300)
    sns.boxplot(data = mydata, linewidth=3, showfliers=False)
    sns.swarmplot(data = mydata, color=".25", size=6)
    if (name == "Average Branch Length") or (name == "Branch Length Per Mito"):
        ax1.set_title(name, size = 24, fontweight='bold')
    elif name == "Max Area / Total Area":
        ax1.set_title(name, size = 24, fontweight='bold')
    else :
        ax1.set_title(name, size = 34, fontweight='bold')

    #ax1.set_aspect(3/max_value)
    if unit == 'area':
        plt.ylabel(r"Area ($\mathbf{\mu m^2}$)", size = 20, fontweight='bold', labelpad=10)
    elif unit == 'length':
        plt.ylabel(r"Length ($\mathbf{\mu m}$)", size = 20, fontweight='bold', labelpad=10)
    elif unit == 'count':
        plt.ylabel(r"Count", size = 20, fontweight='bold', labelpad=10)
    elif unit == 'a.u.':
        plt.ylabel("a.u.", size = 20, fontweight='bold', labelpad=10)
    elif unit == 'ratio':
        plt.ylabel("Ratio (a.u.)", size = 20, fontweight='bold', labelpad=10)
    elif unit == 'normalized count':
        plt.ylabel(r"1 / $\mathbf{\mu m^2}$", size = 20, fontweight='bold', labelpad=10)
    #plt.yticks(size = 25, fontweight='bold')
    plt.xticks([0., 1., 2., 3.], ["Control", "FCCP", "Oligomycin", "Rotenone"], size = 20, fontweight='bold', rotation=30, ha='right')
    plt.yticks(size = 25, fontweight='bold')

    print(f'Levena for {name} = {stats.levene(mydata[0], mydata[1], mydata[2], mydata[3])}')
    print(f'ANOVA for {name} = {stats.f_oneway(mydata[0], mydata[1], mydata[2], mydata[3])}')

    # 0 vs 1
    pv = stats.ttest_ind(mydata[0], mydata[1], equal_var = False).pvalue
    print(f'P-value between group 0 and 1: {pv}')
    if pv <= 0.05:
        bar_start = max_value * 1.04
        barh = max_value * 0.02
        left_x = 0
        right_x = 0.95
        barx = [left_x, left_x, right_x, right_x]
        bary = [bar_start, bar_start+barh, bar_start+barh, bar_start]
        mid = ((left_x+right_x)/2, bar_start+barh)
        plt.plot(barx, bary, c='black', linewidth=5)
        kwargs = dict(ha='center', va='bottom', fontsize='13')
        plt.text(*mid, formatpv(pv), **kwargs)

    # 1 vs 2
    pv = stats.ttest_ind(mydata[1], mydata[2], equal_var = False).pvalue
    print(f'P-value between group 1 and 2: {pv}')
    if pv <= 0.05:
        bar_start = max_value * 1.04
        barh = max_value * 0.02
        left_x = 1.05
        right_x = 1.95
        barx = [left_x, left_x, right_x, right_x]
        bary = [bar_start, bar_start+barh, bar_start+barh, bar_start]
        mid = ((left_x+right_x)/2, bar_start+barh)
        plt.plot(barx, bary, c='black', linewidth=5)
        kwargs = dict(ha='center', va='bottom', fontsize='13')
        plt.text(*mid, formatpv(pv), **kwargs)

    # 2 vs 3
    pv = stats.ttest_ind(mydata[2], mydata[3], equal_var = False).pvalue
    print(f'P-value between group 2 and 3: {pv}')
    if pv <= 0.05:
        bar_start = max_value * 1.04
        barh = max_value * 0.02
        left_x = 2.05
        right_x = 2.95
        barx = [left_x, left_x, right_x, right_x]
        bary = [bar_start, bar_start+barh, bar_start+barh, bar_start]
        mid = ((left_x+right_x)/2, bar_start+barh)
        plt.plot(barx, bary, c='black', linewidth=5)
        kwargs = dict(ha='center', va='bottom', fontsize='13')
        plt.text(*mid, formatpv(pv), **kwargs)

    # 0 vs 2
    pv = stats.ttest_ind(mydata[0], mydata[2], equal_var = False).pvalue
    print(f'P-value between group 0 and 2: {pv}')
    if pv <= 0.05:
        bar_start = max_value * 1.12
        barh = max_value * 0.02
        left_x = 0
        right_x = 1.95
        barx = [left_x, left_x, right_x, right_x]
        bary = [bar_start, bar_start+barh, bar_start+barh, bar_start]
        mid = ((left_x+right_x)/2, bar_start+barh)
        plt.plot(barx, bary, c='black', linewidth=5)
        kwargs = dict(ha='center', va='bottom', fontsize='13')
        plt.text(*mid, formatpv(pv), **kwargs)

    # 1 vs 3
    pv = stats.ttest_ind(mydata[1], mydata[3], equal_var = False).pvalue
    print(f'P-value between group 1 and 3: {pv}')
    if pv <= 0.05:
        bar_start = max_value * 1.20
        barh = max_value * 0.02
        left_x = 1
        right_x = 2.95
        barx = [left_x, left_x, right_x, right_x]
        bary = [bar_start, bar_start+barh, bar_start+barh, bar_start]
        mid = ((left_x+right_x)/2, bar_start+barh)
        plt.plot(barx, bary, c='black', linewidth=5)
        kwargs = dict(ha='center', va='bottom', fontsize='13')
        plt.text(*mid, formatpv(pv), **kwargs)


    # 0 vs 3
    pv = stats.ttest_ind(mydata[0], mydata[3], equal_var = False).pvalue
    print(f'P-value between group 0 and 3: {pv}')
    if pv <= 0.05:
        bar_start = max_value * 1.28
        barh = max_value * 0.02
        left_x = 0
        right_x = 2.95
        barx = [left_x, left_x, right_x, right_x]
        bary = [bar_start, bar_start+barh, bar_start+barh, bar_start]
        mid = ((left_x+right_x)/2, bar_start+barh)
        plt.plot(barx, bary, c='black', linewidth=5)
        kwargs = dict(ha='center', va='bottom', fontsize='13')
        plt.text(*mid, formatpv(pv), **kwargs)

    plt.tight_layout()
    if name == "Max Area / Total Area" :
        plt.savefig(f'{savedir}/{group}/MaxTotal.tif', dpi=300, format="tiff", pil_kwargs={"compression": "tiff_lzw"})
    elif name == "Deg 1 / Deg 3":
        plt.savefig(f'{savedir}/{group}/Deg1_to_Deg3.tif', dpi=300, format="tiff", pil_kwargs={"compression": "tiff_lzw"})
    else:
        plt.savefig(f'{savedir}/{group}/{name}.tif', dpi=300, format="tiff", pil_kwargs={"compression": "tiff_lzw"})
    plt.gcf()

### Data from Analyze Particle Summary

In [8]:
# define the directory of the csv
# APS = Analyze Particle Summary
APS_0X_dir = './data/Analyze_Particle_Summary/Control'
APS_05X_dir = './data/Analyze_Particle_Summary/FCCP'
APS_1X_dir = './data/Analyze_Particle_Summary/Oligomycin'
APS_2X_dir = './data/Analyze_Particle_Summary/Rotenone'

In [9]:
# get the filename list
file_name_APS_0X = []
file_name_APS_05X = []
file_name_APS_1X = []
file_name_APS_2X = []
for filename in os.listdir(APS_0X_dir):
    if filename.endswith(".csv"):
        file_name_APS_0X.append(filename)
for filename in os.listdir(APS_05X_dir):
    if filename.endswith(".csv"):
        file_name_APS_05X.append(filename)
for filename in os.listdir(APS_1X_dir):
    if filename.endswith(".csv"):
        file_name_APS_1X.append(filename)
for filename in os.listdir(APS_2X_dir):
    if filename.endswith(".csv"):
        file_name_APS_2X.append(filename)

In [11]:
tox = ["Control", "FCCP", "Oligomycin", "Rotenone"]
file_name_APS_list = [file_name_APS_0X, file_name_APS_05X, file_name_APS_1X, file_name_APS_2X]
Count = [[],[],[],[]]
nCount = [[],[],[],[]]
TotalArea = [[],[],[],[]]
AverageArea = [[],[],[],[]]
Perimeter = [[],[],[],[]]
FormFactor = [[],[],[],[]]

for i in range(len(tox)):
    for j in range(len(file_name_APS_list[i])):
        df = pd.read_csv(f'./data/Analyze_Particle_Summary/{tox[i]}/{file_name_APS_list[i][j]}', index_col=0)
        Count[i].append(float(df['Count'].iloc[0]))
        TotalArea[i].append(float(df['Total Area'].iloc[0]))
        AverageArea[i].append(float(df['Average Size'].iloc[0]))
        Perimeter[i].append(float(df['Perim.'].iloc[0]))
        FormFactor[i].append(np.reciprocal(float(df['Circ.'].iloc[0])))
        nCount[i].append(float(df['Count'].iloc[0])/float(df['Total Area'].iloc[0]))

### Data from Analyze Particle Results

In [12]:
# define the directory of the csv
# APR = Analyze Particle Results
APR_0X_dir = './data/Analyze_Particle_Results/Control'
APR_05X_dir = './data/Analyze_Particle_Results/FCCP'
APR_1X_dir = './data/Analyze_Particle_Results/Oligomycin'
APR_2X_dir = './data/Analyze_Particle_Results/Rotenone'

# get the filename list
file_name_APR_0X = []
file_name_APR_05X = []
file_name_APR_1X = []
file_name_APR_2X = []

for filename in os.listdir(APR_0X_dir):
    if filename.endswith(".csv"):
        file_name_APR_0X.append(filename)
for filename in os.listdir(APR_05X_dir):
    if filename.endswith(".csv"):
        file_name_APR_05X.append(filename)
for filename in os.listdir(APR_1X_dir):
    if filename.endswith(".csv"):
        file_name_APR_1X.append(filename)
for filename in os.listdir(APR_2X_dir):
    if filename.endswith(".csv"):
        file_name_APR_2X.append(filename)

In [13]:
tox = ["Control", "FCCP", "Oligomycin", "Rotenone"]
file_name_APR_list = [file_name_APR_0X, file_name_APR_05X, file_name_APR_1X, file_name_APR_2X]
AspectRatio = [[],[],[],[]]
MaxArea = [[],[],[],[]]

for i in range(len(tox)):
    for j in range(len(file_name_APS_list[i])):
        df = pd.read_csv(f'./data/Analyze_Particle_Results/{tox[i]}/{file_name_APR_list[i][j]}', index_col=0)
        AspectRatio[i].append(df['AR'].mean())
        MaxArea[i].append(df.loc[df['Area'].idxmax()]['Area'])

In [14]:
total_area_0X_np = np.array(TotalArea[0])
total_area_05X_np = np.array(TotalArea[1])
total_area_1X_np = np.array(TotalArea[2])
total_area_2X_np = np.array(TotalArea[3])

max_area_0X_np = np.array(MaxArea[0])
max_area_05X_np = np.array(MaxArea[1])
max_area_1X_np = np.array(MaxArea[2])
max_area_2X_np = np.array(MaxArea[3])

temp_data_0X = max_area_0X_np/total_area_0X_np
temp_data_05X = max_area_05X_np/total_area_05X_np
temp_data_1X = max_area_1X_np/total_area_1X_np
temp_data_2X = max_area_2X_np/total_area_2X_np
MaxTotal = [temp_data_0X, temp_data_05X, temp_data_1X, temp_data_2X]

### Data From Skeleton Results

In [15]:
# define the directory of the csv
# APR = Analyze Particle Results
SR_0X_dir = './data/Skeleton_Results/Control'
SR_05X_dir = './data/Skeleton_Results/FCCP'
SR_1X_dir = './data/Skeleton_Results/Oligomycin'
SR_2X_dir = './data/Skeleton_Results/Rotenone'

# get the filename list
file_name_SR_0X = []
file_name_SR_05X = []
file_name_SR_1X = []
file_name_SR_2X = []

for filename in os.listdir(SR_0X_dir):
    if filename.endswith(".csv"):
        file_name_SR_0X.append(filename)
for filename in os.listdir(SR_05X_dir):
    if filename.endswith(".csv"):
        file_name_SR_05X.append(filename)
for filename in os.listdir(SR_1X_dir):
    if filename.endswith(".csv"):
        file_name_SR_1X.append(filename)
for filename in os.listdir(SR_2X_dir):
    if filename.endswith(".csv"):
        file_name_SR_2X.append(filename)

tox = ["Control", "FCCP", "Oligomycin", "Rotenone"]
file_name_SR_list = [file_name_SR_0X, file_name_SR_05X, file_name_SR_1X, file_name_SR_2X]

TotalBranches = [[],[],[],[]]
AvgBranches = [[],[],[],[]]
ABL = [[],[],[],[]]
BLpM = [[],[],[],[]]
TotalDeg1 = [[],[],[],[]]
TotalDeg2 = [[],[],[],[]]
TotalDeg3 = [[],[],[],[]]
AvgDeg1 = [[],[],[],[]]
AvgDeg2 = [[],[],[],[]]
AvgDeg3 = [[],[],[],[]]
AvgDeg = [[],[],[],[]]




for i in range(len(tox)):
    for j in range(len(file_name_SR_list[i])):
        df = pd.read_csv(f'./data/Skeleton_Results/{tox[i]}/{file_name_SR_list[i][j]}', index_col=0)
        # Calculate deg 2
        fake_deg2 = np.zeros(df[df.columns[0]].count())
        deg2 = np.zeros(df[df.columns[0]].count())
        real_ABL = np.sum(df['# Branches'] * df['Average Branch Length']) / np.sum(df['# Branches'])
        for k in range(len(deg2)):
            fake_deg2[k] = (df['# Branches'][k+1] * df['Average Branch Length'][k+1]) / real_ABL
            dif = round(fake_deg2[k] - (df['# End-point voxels'][k+1] + df['# Junctions'][k+1])) # dif = #2 - (#1 + #3), 0)
            if dif <= 0:
                deg2[k] = 0
            elif dif > 0 :
                deg2[k] = dif
        df['fake_Deg2'] = fake_deg2
        df['Deg2'] = deg2

        TotalBranches[i].append(df['# Branches'].sum())
        AvgBranches[i].append(df['# Branches'].mean())
        ABL[i].append(df['Average Branch Length'].mean())
        BLpM[i].append((df['# Branches'] * df['Average Branch Length']).mean())
        TotalDeg1[i].append(df['# End-point voxels'].sum())
        TotalDeg2[i].append(df['Deg2'].sum())
        TotalDeg3[i].append(df['# Junctions'].sum())
        AvgDeg1[i].append(df['# End-point voxels'].mean())
        AvgDeg2[i].append(df['Deg2'].mean())
        AvgDeg3[i].append(df['# Junctions'].mean())
        avgdeg = (df['# End-point voxels'].sum() + 2*df['Deg2'].sum() + 3*df['# Junctions'].sum()) / (df['# End-point voxels'].sum() + df['Deg2'].sum() + df['# Junctions'].sum())
        AvgDeg[i].append(avgdeg)

TotalDeg1_0X_np = np.array(TotalDeg1[0])
TotalDeg1_05X_np = np.array(TotalDeg1[1])
TotalDeg1_1X_np = np.array(TotalDeg1[2])
TotalDeg1_2X_np = np.array(TotalDeg1[3])

TotalDeg2_0X_np = np.array(TotalDeg2[0])
TotalDeg2_05X_np = np.array(TotalDeg2[1])
TotalDeg2_1X_np = np.array(TotalDeg2[2])
TotalDeg2_2X_np = np.array(TotalDeg2[3])

TotalDeg3_0X_np = np.array(TotalDeg3[0])
TotalDeg3_05X_np = np.array(TotalDeg3[1])
TotalDeg3_1X_np = np.array(TotalDeg3[2])
TotalDeg3_2X_np = np.array(TotalDeg3[3])

# Deg1_to_Deg3
temp_data_0X = TotalDeg1_0X_np/TotalDeg3_0X_np
temp_data_05X = TotalDeg1_05X_np/TotalDeg3_05X_np
temp_data_1X = TotalDeg1_1X_np/TotalDeg3_1X_np
temp_data_2X = TotalDeg1_2X_np/TotalDeg3_2X_np
Deg1_to_Deg3 = [temp_data_0X, temp_data_05X, temp_data_1X, temp_data_2X]

# nDeg1
temp_data_0X = TotalDeg1_0X_np/total_area_0X_np
temp_data_05X = TotalDeg1_05X_np/total_area_05X_np
temp_data_1X = TotalDeg1_1X_np/total_area_1X_np
temp_data_2X = TotalDeg1_2X_np/total_area_2X_np
nDeg1 = [temp_data_0X, temp_data_05X, temp_data_1X, temp_data_2X]

#nDeg2
temp_data_0X = TotalDeg2_0X_np/total_area_0X_np
temp_data_05X = TotalDeg2_05X_np/total_area_05X_np
temp_data_1X = TotalDeg2_1X_np/total_area_1X_np
temp_data_2X = TotalDeg2_2X_np/total_area_2X_np
nDeg2 = [temp_data_0X, temp_data_05X, temp_data_1X, temp_data_2X]

#nDeg3
temp_data_0X = TotalDeg3_0X_np/total_area_0X_np
temp_data_05X = TotalDeg3_05X_np/total_area_05X_np
temp_data_1X = TotalDeg3_1X_np/total_area_1X_np
temp_data_2X = TotalDeg3_2X_np/total_area_2X_np
nDeg3 = [temp_data_0X, temp_data_05X, temp_data_1X, temp_data_2X]

In [18]:
Plot4_seaborn(Count, "Count", unit = "count", group = "particle")

Levena for Count = LeveneResult(statistic=np.float64(2.401756741355261), pvalue=np.float64(0.07565686137936803))
ANOVA for Count = F_onewayResult(statistic=np.float64(9.893762399788413), pvalue=np.float64(1.8455778287326068e-05))
P-value between group 0 and 1: 0.00346841028180536
P-value between group 1 and 2: 4.6447578271244865e-05
P-value between group 2 and 3: 0.0041001617868454835
P-value between group 0 and 2: 0.02561208986447697
P-value between group 1 and 3: 0.10578636777553477
P-value between group 0 and 3: 0.29954277814277885


In [19]:
Plot4_seaborn(nCount, "Normalized Count", unit = "count", group = "particle")

Levena for Normalized Count = LeveneResult(statistic=np.float64(10.446539616815631), pvalue=np.float64(1.0675880979446341e-05))
ANOVA for Normalized Count = F_onewayResult(statistic=np.float64(26.314192437038635), pvalue=np.float64(2.928832297616059e-11))
P-value between group 0 and 1: 5.152946777481957e-05
P-value between group 1 and 2: 0.07350876381351618
P-value between group 2 and 3: 3.4801341754996943e-12
P-value between group 0 and 2: 4.040925060739468e-06
P-value between group 1 and 3: 2.4738372654622534e-07
P-value between group 0 and 3: 0.0011298854379027706


In [20]:
Plot4_seaborn(TotalArea, "Total Area", unit = "area", group = "particle")

Levena for Total Area = LeveneResult(statistic=np.float64(2.5381231728896414), pvalue=np.float64(0.06420108164422307))
ANOVA for Total Area = F_onewayResult(statistic=np.float64(9.864126078234692), pvalue=np.float64(1.9010093060988534e-05))
P-value between group 0 and 1: 5.1935943245557734e-05
P-value between group 1 and 2: 2.1219760041174346e-05
P-value between group 2 and 3: 0.28827082693142897
P-value between group 0 and 2: 0.5997130085097484
P-value between group 1 and 3: 0.0005078388273830519
P-value between group 0 and 3: 0.5339820634852854


In [21]:
Plot4_seaborn(AverageArea, "Average Area", unit = "area", group = "particle")

Levena for Average Area = LeveneResult(statistic=np.float64(3.6695407322164257), pvalue=np.float64(0.016582307689687))
ANOVA for Average Area = F_onewayResult(statistic=np.float64(32.93957718544294), pvalue=np.float64(4.577907389943614e-13))
P-value between group 0 and 1: 3.572500901140919e-05
P-value between group 1 and 2: 0.3810855597828968
P-value between group 2 and 3: 6.622975027292084e-07
P-value between group 0 and 2: 4.417659337091364e-05
P-value between group 1 and 3: 8.024869023579523e-08
P-value between group 0 and 3: 0.0024029549916900414


In [22]:
Plot4_seaborn(Perimeter, "Perimeter", unit = "length", group = "particle")

Levena for Perimeter = LeveneResult(statistic=np.float64(5.050943474311408), pvalue=np.float64(0.003307666357441376))
ANOVA for Perimeter = F_onewayResult(statistic=np.float64(29.15163964894621), pvalue=np.float64(4.623885120876661e-12))
P-value between group 0 and 1: 0.00015638894844651217
P-value between group 1 and 2: 0.4980354923567748
P-value between group 2 and 3: 1.2158388700538019e-06
P-value between group 0 and 2: 3.0131451454339983e-05
P-value between group 1 and 3: 1.2374216624869324e-06
P-value between group 0 and 3: 0.010318151401806037


In [23]:
Plot4_seaborn(FormFactor, "Form Factor", unit = "a.u.", group = "particle")

Levena for Form Factor = LeveneResult(statistic=np.float64(7.147434908216651), pvalue=np.float64(0.00032002771927655766))
ANOVA for Form Factor = F_onewayResult(statistic=np.float64(21.53238768138094), pvalue=np.float64(8.489467692671564e-10))
P-value between group 0 and 1: 0.4058013527128045
P-value between group 1 and 2: 2.0846726419259427e-10
P-value between group 2 and 3: 1.7330754097237464e-07
P-value between group 0 and 2: 3.8732169331789836e-05
P-value between group 1 and 3: 0.0637269977434842
P-value between group 0 and 3: 0.030024427196823293


In [24]:
Plot4_seaborn(AspectRatio, "Aspect Ratio", unit = "a.u.", group = "particle")

Levena for Aspect Ratio = LeveneResult(statistic=np.float64(8.833660427295989), pvalue=np.float64(5.404308142593828e-05))
ANOVA for Aspect Ratio = F_onewayResult(statistic=np.float64(27.95983317256117), pvalue=np.float64(9.914196094708825e-12))
P-value between group 0 and 1: 3.3715271517573377e-05
P-value between group 1 and 2: 0.38047135464276205
P-value between group 2 and 3: 8.655449548581027e-09
P-value between group 0 and 2: 7.945477999672803e-06
P-value between group 1 and 3: 2.2058128173296627e-08
P-value between group 0 and 3: 0.3284062384721933


In [25]:
Plot4_seaborn(MaxTotal, "Max Area / Total Area", unit = "ratio", group = "particle")

Levena for Max Area / Total Area = LeveneResult(statistic=np.float64(3.3800235629551265), pvalue=np.float64(0.023395455055828605))
ANOVA for Max Area / Total Area = F_onewayResult(statistic=np.float64(3.0519795876471667), pvalue=np.float64(0.03462706046499522))
P-value between group 0 and 1: 0.15121554330829315
P-value between group 1 and 2: 0.023022123479329765
P-value between group 2 and 3: 0.013248863708499307
P-value between group 0 and 2: 0.02759646009562378
P-value between group 1 and 3: 0.7056397218937807
P-value between group 0 and 3: 0.16909980409698958


In [26]:
Plot4_seaborn(TotalBranches, 'Total Branches', unit = 'count', group = "skeleton")

Levena for Total Branches = LeveneResult(statistic=np.float64(0.4568373899406688), pvalue=np.float64(0.713379403121075))
ANOVA for Total Branches = F_onewayResult(statistic=np.float64(1.8985929934125672), pvalue=np.float64(0.1385580691157818))
P-value between group 0 and 1: 0.032883453638763716
P-value between group 1 and 2: 0.04801173513712264
P-value between group 2 and 3: 0.8759683059559381
P-value between group 0 and 2: 0.78261149470658
P-value between group 1 and 3: 0.15603818165176814
P-value between group 0 and 3: 0.7089130613606427


In [27]:
Plot4_seaborn(AvgBranches, 'Average Branches', unit = 'count', group = "skeleton")

Levena for Average Branches = LeveneResult(statistic=np.float64(3.5377588473843646), pvalue=np.float64(0.019390389088073792))
ANOVA for Average Branches = F_onewayResult(statistic=np.float64(10.335926149860308), pvalue=np.float64(1.190350763204157e-05))
P-value between group 0 and 1: 0.37284960141184575
P-value between group 1 and 2: 7.904991144567854e-07
P-value between group 2 and 3: 1.4828527781246469e-05
P-value between group 0 and 2: 0.0005886197431700385
P-value between group 1 and 3: 0.9111061310745234
P-value between group 0 and 3: 0.34930488305390905


In [28]:
Plot4_seaborn(ABL, 'Average Branch Length', unit = 'length', group = "skeleton")

Levena for Average Branch Length = LeveneResult(statistic=np.float64(3.804057299925102), pvalue=np.float64(0.01414096400103188))
ANOVA for Average Branch Length = F_onewayResult(statistic=np.float64(50.201648003498796), pvalue=np.float64(6.51620374326256e-17))
P-value between group 0 and 1: 4.86942845524894e-06
P-value between group 1 and 2: 0.983814654245488
P-value between group 2 and 3: 5.271853121954893e-11
P-value between group 0 and 2: 4.392661123686197e-06
P-value between group 1 and 3: 3.8185997761484683e-11
P-value between group 0 and 3: 0.0002485579853257326


In [29]:
Plot4_seaborn(BLpM, 'Branch Length Per Mito', unit = 'length', group = "skeleton")

Levena for Branch Length Per Mito = LeveneResult(statistic=np.float64(6.18598712881479), pvalue=np.float64(0.0009180655891553269))
ANOVA for Branch Length Per Mito = F_onewayResult(statistic=np.float64(24.601878881649593), pvalue=np.float64(9.402721167803396e-11))
P-value between group 0 and 1: 0.0019566364482916684
P-value between group 1 and 2: 0.0208441360934158
P-value between group 2 and 3: 1.0227009351579099e-06
P-value between group 0 and 2: 4.910478186214935e-05
P-value between group 1 and 3: 6.108190761709371e-06
P-value between group 0 and 3: 0.019812108356078828


In [30]:
Plot4_seaborn(TotalDeg1, 'Total Deg 1', unit = 'count', group = "skeleton")

Levena for Total Deg 1 = LeveneResult(statistic=np.float64(1.0957221295746675), pvalue=np.float64(0.35733987143231327))
ANOVA for Total Deg 1 = F_onewayResult(statistic=np.float64(5.564822031184559), pvalue=np.float64(0.0018421791474564513))
P-value between group 0 and 1: 0.01262567064434905
P-value between group 1 and 2: 0.0005291716506665745
P-value between group 2 and 3: 0.03669124748381144
P-value between group 0 and 2: 0.12655301625344462
P-value between group 1 and 3: 0.17785935201050534
P-value between group 0 and 3: 0.40515422443507765


In [31]:
Plot4_seaborn(TotalDeg2, 'Total Deg 2', unit = 'count', group = "skeleton")

Levena for Total Deg 2 = LeveneResult(statistic=np.float64(1.679027115235726), pvalue=np.float64(0.1801621702644636))
ANOVA for Total Deg 2 = F_onewayResult(statistic=np.float64(5.362394412192066), pvalue=np.float64(0.002317593983659467))
P-value between group 0 and 1: 0.0005376455318667336
P-value between group 1 and 2: 0.0015502750665346864
P-value between group 2 and 3: 0.6436515825140094
P-value between group 0 and 2: 0.7534259734624877
P-value between group 1 and 3: 0.023342783343081236
P-value between group 0 and 3: 0.4642757553516843


In [32]:
Plot4_seaborn(TotalDeg3, 'Total Deg 3', unit = 'count', group = "skeleton")

Levena for Total Deg 3 = LeveneResult(statistic=np.float64(1.7120755730316104), pvalue=np.float64(0.173198530224424))
ANOVA for Total Deg 3 = F_onewayResult(statistic=np.float64(2.6166689390833175), pvalue=np.float64(0.058409801766667366))
P-value between group 0 and 1: 0.11388097619752975
P-value between group 1 and 2: 0.48595310972732375
P-value between group 2 and 3: 0.042224230509082326
P-value between group 0 and 2: 0.018475360218474454
P-value between group 1 and 3: 0.14362381030620447
P-value between group 0 and 3: 0.9082057499992617


In [33]:
Plot4_seaborn(AvgDeg1, 'Average Deg 1', unit = 'count', group = "skeleton")

Levena for Average Deg 1 = LeveneResult(statistic=np.float64(2.6282903824796207), pvalue=np.float64(0.057598678565400636))
ANOVA for Average Deg 1 = F_onewayResult(statistic=np.float64(12.527626403105483), pvalue=np.float64(1.465134067671843e-06))
P-value between group 0 and 1: 0.04219200676972627
P-value between group 1 and 2: 8.502420610300866e-08
P-value between group 2 and 3: 8.831759969785956e-05
P-value between group 0 and 2: 0.0005678539753960726
P-value between group 1 and 3: 0.2168804254198499
P-value between group 0 and 3: 0.43249455541976034


In [34]:
Plot4_seaborn(AvgDeg2, 'Average Deg 2', unit = 'count', group = "skeleton")

Levena for Average Deg 2 = LeveneResult(statistic=np.float64(3.318460438069838), pvalue=np.float64(0.025177936659869206))
ANOVA for Average Deg 2 = F_onewayResult(statistic=np.float64(5.172703999790071), pvalue=np.float64(0.002877212101556875))
P-value between group 0 and 1: 0.09246279496775267
P-value between group 1 and 2: 0.1705095539595084
P-value between group 2 and 3: 0.00018973758392295216
P-value between group 0 and 2: 0.00038967287101180937
P-value between group 1 and 3: 0.08299638923350733
P-value between group 0 and 3: 0.9608398589659375


In [35]:
Plot4_seaborn(AvgDeg3, 'Average Deg 3', unit = 'count', group = "skeleton")

Levena for Average Deg 3 = LeveneResult(statistic=np.float64(3.9358684044806957), pvalue=np.float64(0.012103041235852442))
ANOVA for Average Deg 3 = F_onewayResult(statistic=np.float64(9.491760594813847), pvalue=np.float64(2.7631045319463646e-05))
P-value between group 0 and 1: 0.5937739845306971
P-value between group 1 and 2: 2.3922719924353842e-06
P-value between group 2 and 3: 1.1847417127894812e-05
P-value between group 0 and 2: 0.0008568841016767979
P-value between group 1 and 3: 0.5440710919955039
P-value between group 0 and 3: 0.31548234065148606


In [36]:
Plot4_seaborn(nDeg1, 'Normalized Deg 1', unit = 'count', group = "skeleton")

Levena for Normalized Deg 1 = LeveneResult(statistic=np.float64(16.236184107006864), pvalue=np.float64(5.547277312720028e-08))
ANOVA for Normalized Deg 1 = F_onewayResult(statistic=np.float64(40.15581793067866), pvalue=np.float64(8.396503554200434e-15))
P-value between group 0 and 1: 1.3919865491119005e-06
P-value between group 1 and 2: 0.0003480580750777661
P-value between group 2 and 3: 3.781610252489695e-13
P-value between group 0 and 2: 6.7822733219065035e-06
P-value between group 1 and 3: 2.114721135756083e-08
P-value between group 0 and 3: 5.0404010711065865e-05


In [37]:
Plot4_seaborn(nDeg2, 'Normalized Deg 2', unit = 'count', group = "skeleton")

C:\Users\sosiristseng\AppData\Local\Temp\ipykernel_20924\3999241140.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig1, ax1 = plt.subplots(figsize=(6, 9), dpi=300)


Levena for Normalized Deg 2 = LeveneResult(statistic=np.float64(7.033383590537953), pvalue=np.float64(0.0003620904338747146))
ANOVA for Normalized Deg 2 = F_onewayResult(statistic=np.float64(8.251806901779323), pvalue=np.float64(9.88561646765326e-05))
P-value between group 0 and 1: 0.023907099565403128
P-value between group 1 and 2: 0.02897750147813634
P-value between group 2 and 3: 0.0011103926997533783
P-value between group 0 and 2: 0.8666685702126112
P-value between group 1 and 3: 0.00022523790090678178
P-value between group 0 and 3: 0.0017763895044667758


In [38]:
Plot4_seaborn(nDeg3, 'Normalized Deg 3',  unit = 'count', group = "skeleton")

Levena for Normalized Deg 3 = LeveneResult(statistic=np.float64(5.425990029271337), pvalue=np.float64(0.002156023607686799))
ANOVA for Normalized Deg 3 = F_onewayResult(statistic=np.float64(29.94947584285445), pvalue=np.float64(2.80213435298105e-12))
P-value between group 0 and 1: 4.8700548813948e-06
P-value between group 1 and 2: 2.2167934132941423e-08
P-value between group 2 and 3: 0.025535878016185544
P-value between group 0 and 2: 0.030220119500437397
P-value between group 1 and 3: 4.671031468962532e-07
P-value between group 0 and 3: 0.41048298697044694


In [39]:
Plot4_seaborn(AvgDeg, 'Average Degree', unit = 'count', group = "skeleton")

Levena for Average Degree = LeveneResult(statistic=np.float64(3.647138847800297), pvalue=np.float64(0.017028722313235514))
ANOVA for Average Degree = F_onewayResult(statistic=np.float64(11.185019487846269), pvalue=np.float64(5.206863688616042e-06))
P-value between group 0 and 1: 0.8642355843606242
P-value between group 1 and 2: 1.7239821550307552e-06
P-value between group 2 and 3: 2.084706023030623e-07
P-value between group 0 and 2: 0.0006914998538903521
P-value between group 1 and 3: 0.13158006021439597
P-value between group 0 and 3: 0.19171601056985202


In [40]:
Plot4_seaborn(Deg1_to_Deg3, "Deg 1 / Deg 3", unit = 'ratio', group = "skeleton")

Levena for Deg 1 / Deg 3 = LeveneResult(statistic=np.float64(1.575002463032881), pvalue=np.float64(0.20390073378332904))
ANOVA for Deg 1 / Deg 3 = F_onewayResult(statistic=np.float64(3.183669408171367), pvalue=np.float64(0.029576923229950428))
P-value between group 0 and 1: 0.25976774097154226
P-value between group 1 and 2: 3.812473926845105e-06
P-value between group 2 and 3: 5.081935670088848e-07
P-value between group 0 and 2: 0.43948690662930723
P-value between group 1 and 3: 0.19998109712869996
P-value between group 0 and 3: 0.1642318495299295
